In [1]:
pip install Backtesting

     |████████████████████████████████| 174kB 4.3MB/s 
  Created wheel for Backtesting: filename=Backtesting-0.2.4-cp36-none-any.whl size=169967 sha256=11fed42a2f80bc8245f4f0a7b501d9a024cd854db694a9425117949c20dcebe3
  Stored in directory: /root/.cache/pip/wheels/11/bf/97/adb7eb1803f631b1c0358c15d8b8e60e71b647e632f472f960
Successfully built Backtesting


In [2]:
# Configurando Yahoo Finance
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf

     |████████████████████████████████| 5.5MB 6.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=0503f02b28e98fde54b042877e4cd7bf2c1245a1dd6debd070a1a7d513a5d1d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-s4jb405d/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import random

In [ ]:
ticker = "WEGE3.SA"

dados_yahoo = yf.download(tickers=ticker, period='1y')
dados_yahoo

In [ ]:
dados_yahoo_index = dados_yahoo.index.tolist()
dados_yahoo['Close'].values.tolist()

In [ ]:
import pandas as pd

acoes = {'Open': dados_yahoo['Open'].values.tolist(),
        'High': dados_yahoo['High'].values.tolist(),
        'Low': dados_yahoo['Low'].values.tolist(),
        'Close': dados_yahoo['Close'].values.tolist(),
        'Volume': dados_yahoo['Volume'].values.tolist()
        }

df = pd.DataFrame(acoes, columns = ['Open', 'High', 'Low', 'Close', 'Volume'], index=dados_yahoo_index)

df

In [ ]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

    

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 8
    n2 = 34
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest
df=df.dropna()
bt = Backtest(df, SmaCross, cash=10000, commission=0.002)
stats = bt.run()
stats

In [ ]:
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 150, 5),
                    n2=range(5, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
.stats.tail()

In [ ]:
stats['_equity_curve']

In [ ]:
stats['_trades'] 